In [1]:

# TODO: for each ds, load it, filter relevant columns and join on nft_address

In [2]:
import os
import sqlite3

import matplotlib.pyplot as plt
import numpy as np
import nfts.dataset

import pandas as pd
import math

In [66]:
dirname = os.path.dirname(os.path.abspath("__file__"))
DATASET_PATH = os.path.join(dirname, "nfts.sqlite")
ds = nfts.dataset.FromSQLite(DATASET_PATH)

nfts_df = ds.load_dataframe("nfts")
nfts_df.rename(columns={'address':"nft_address"}, inplace=True)
nfts_df.head()

,nft_address,name,symbol
0,0xF4ee95274741437636e748DdAc70818B4ED7d043,The Doge Pound,DOGGY
1,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC
2,0x7403AC30DE7309a0bF019cdA8EeC034a5507cbB3,Characters,CHAR
3,0x511372B44231a31527025a3D273C1dc0a83D77aF,Maps Restored,MAP
4,0x2E956Ed3D7337F4Ed4316A6e8F2EdF74BF84bb54,Neon Junkies,JUNKIES


Collections with most contracts

In [4]:
collections_df = nfts_df.groupby('name', as_index=False).size().rename(columns={'size':'num_addresses'})
collections_df.sort_values('num_addresses', ascending=False, inplace=True)

collections_df.head()

,name,num_addresses
4894,NFT Basket,68
0,,19
4933,NFT-LOTTERY,16
7406,Tinlake Loan Token,13
568,Artiikorea,9


# Tables Joining

- nfts
    - address
- mints
    - transaction_value
    - timestamp
    - block_number ???
    - to_address ???
- transfers
    - transaction_value
    - block_number ???
- current_owners
    - owner
- current_market_values
    - market_value
- transfer_statistics_by_address
    - number of transfers

Tables loading and projection

In [5]:
mints_df = ds.load_dataframe("mints")
mints_df = mints_df[['nft_address', 'transaction_value', 'timestamp']]

transfers_df = ds.load_dataframe("transfers")
transfers_df = transfers_df[['nft_address', 'transaction_value']]

current_owners_df = ds.load_dataframe("current_owners")
current_owners_df = current_owners_df[['nft_address', 'owner']]

current_market_values_df = ds.load_dataframe("current_market_values")
current_market_values_df = current_market_values_df[['nft_address', 'market_value']]


In [32]:

# TODO: remove cell

print(list(nfts_df.keys()), "nfts")

print(list(mints_df.keys()), "mints")
print(list(transfers_df.keys()), "transfers")
print(list(current_owners_df.keys()), "owners")
print(list(current_market_values_df.keys()), "values")

['nft_address', 'name', 'symbol'] nfts
['nft_address', 'transaction_value', 'timestamp'] mints
['nft_address', 'transaction_value'] transfers
['nft_address', 'owner'] owners
['nft_address', 'market_value'] values


In [27]:

# TODO: for each nft_address, number of transfers (see transfers.ipynb)

print("nft unique addresses:\t", nfts_df["nft_address"].nunique(), " / ", len(nfts_df))
print("Mint unique addresses:\t", mints_df["nft_address"].nunique(), " / ", len(mints_df))
print("Transfers unique addresses:\t", transfers_df["nft_address"].nunique(), " / ", len(transfers_df))
print("Owners unique addresses:\t", current_owners_df["nft_address"].nunique(), " / ", len(current_owners_df))
print("Values unique addresses:\t", current_market_values_df["nft_address"].nunique(), " / ", len(current_market_values_df))


nft unique addresses:	 9388  /  9388
Mint unique addresses:	 7647  /  6667282
Transfers unique addresses:	 6701  /  4514729
Owners unique addresses:	 9292  /  7020950
Values unique addresses:	 9292  /  7020950


In [58]:
nfts_df.drop(['avg_mints_transaction_value'], axis=1, inplace=True)

In [72]:

# TODO: change column names before merge

# for mints, take the average transaction_value and the difference in timestamp
mints_grouped_df = mints_df.groupby("nft_address", as_index=False).aggregate({'transaction_value': 'mean', 'timestamp': lambda x: x.max() - x.min()}) 
nfts_merged_df = nfts_df.merge(mints_grouped_df, on="nft_address", how="left")

# for transfers
transfers_grouped_df = transfers_df.groupby("nft_address", as_index=False).mean() 
nfts_merged_df = nfts_df.merge(transfers_grouped_df, on="nft_address", how="left")

nfts_merged_df.head()

,nft_address,name,symbol,transaction_value
0,0xF4ee95274741437636e748DdAc70818B4ED7d043,The Doge Pound,DOGGY,8.094974e+17
1,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC,1.782388e+18
2,0x7403AC30DE7309a0bF019cdA8EeC034a5507cbB3,Characters,CHAR,1.448103e+17
3,0x511372B44231a31527025a3D273C1dc0a83D77aF,Maps Restored,MAP,1.072172e+17
4,0x2E956Ed3D7337F4Ed4316A6e8F2EdF74BF84bb54,Neon Junkies,JUNKIES,8.824530e+16


# Clustering

# Conclusions